In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
# Create tool

@tool
def multiply(a:int, b:int) -> int:
    """Given two numbers a and b, this tool returns their product"""
    return a * b

In [5]:
print(multiply.invoke({'a':2, 'b':3}))

6


In [6]:
multiply.description

'Given two numbers a and b, this tool returns their product'

In [7]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

### Tool Binding

In [8]:
llm = ChatOpenAI()

In [9]:
llm_with_tools = llm.bind_tools([multiply])
llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001C302472600>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001C302472A50>, root_client=<openai.OpenAI object at 0x000001C301A60950>, root_async_client=<openai.AsyncOpenAI object at 0x000001C302472630>, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Given two numbers a and b, this tool returns their product', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [10]:
llm_with_tools.invoke('Hey how are ya?')

AIMessage(content="Hello! I'm here and ready to assist you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 59, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b26702a7-91de-44b3-bcad-428ba665a847-0', usage_metadata={'input_tokens': 59, 'output_tokens': 20, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Tool Calling

In [11]:
llm_with_tools.invoke('Can you multiply 3 with 8?')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_G3Pipnrj0UPNYoHH0rTnaW5n', 'function': {'arguments': '{"a":3,"b":8}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-17702f3b-ce08-4b4c-b871-8173486bfddb-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 8}, 'id': 'call_G3Pipnrj0UPNYoHH0rTnaW5n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 18, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
llm_with_tools.invoke('Can you multiply 3 with 8?').tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 8},
 'id': 'call_p8BCtByoiPDRPXcQf3xNuPEv',
 'type': 'tool_call'}

### Tool Execution

In [13]:
result = llm_with_tools.invoke('Can you multiply 3 with 8?')
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 8},
 'id': 'call_Xr9SOe3fdr5DG5De3IyI5Xz0',
 'type': 'tool_call'}

In [14]:
query = HumanMessage('Can you multiply 3 with 8')

In [15]:
messages = [query]

In [16]:
messages

[HumanMessage(content='Can you multiply 3 with 8', additional_kwargs={}, response_metadata={})]

In [17]:
result = llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LJNZ5qNAHXaY9g5a1fyLj49b', 'function': {'arguments': '{"a":3,"b":8}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d6c5ea63-5d6d-414a-ba14-e50221ab74e4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 8}, 'id': 'call_LJNZ5qNAHXaY9g5a1fyLj49b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_tokens': 18, 'total_tokens': 80, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
messages.append(result)

In [19]:
messages

[HumanMessage(content='Can you multiply 3 with 8', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LJNZ5qNAHXaY9g5a1fyLj49b', 'function': {'arguments': '{"a":3,"b":8}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d6c5ea63-5d6d-414a-ba14-e50221ab74e4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 8}, 'id': 'call_LJNZ5qNAHXaY9g5a1fyLj49b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_tokens': 18, 'total_tokens': 80, 'input_token_details': {'audio': 0,

In [20]:
tool_result = multiply.invoke(result.tool_calls[0])

In [21]:
messages.append(tool_result)

In [22]:
messages

[HumanMessage(content='Can you multiply 3 with 8', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LJNZ5qNAHXaY9g5a1fyLj49b', 'function': {'arguments': '{"a":3,"b":8}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d6c5ea63-5d6d-414a-ba14-e50221ab74e4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 8}, 'id': 'call_LJNZ5qNAHXaY9g5a1fyLj49b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_tokens': 18, 'total_tokens': 80, 'input_token_details': {'audio': 0,